## Implementación del modelo en tiempo real

Permite detectar emociones faciales en tiempo real utilizando la cámara web y un modelo previamente entrenado. De forma general hace lo siguiente:

1. **Carga del modelo entrenado:** Se importa un modelo Keras previamente guardado (.keras) especializado en la clasificación de emociones.

2. **Definición de clases:** Se establecen las etiquetas de las emociones que el modelo puede predecir.

3. **Detección de rostros:** Se usa un clasificador Haar de OpenCV para detectar rostros dentro del video en vivo.

4. **Preprocesamiento:** Cada rostro detectado se redimensiona y normaliza para coincidir con el formato esperado por el modelo.

5. **Predicción y suavizado:** Se predice la emoción del rostro. Se implementa un suavizado con historial para evitar saltos bruscos entre emociones.

6. **Visualización:** Se muestra la emoción detectada sobre el rostro en el video, junto con el porcentaje de confianza.


<small> **Nota:** _En caso de que quiera probar el programa, el modelo no esta en el repositorio debido a que excede 100 MB._ <small>

In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from collections import deque

# 1. Cargar el modelo entrenado y configuraciones
model = load_model('mejor_modelo_emociones.keras')  
IMG_SIZE = (224, 224)

# 2. Definir las clases
class_names = {
    0: 'Enojo',
    1: 'Disgusto',
    2: 'Miedo',
    3: 'Felicidad',
    4: 'Neutral',
    5: 'Tristeza',
    6: 'Sorpresa'
}

# 3. Configurar detector de rostros
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 4. Configurar suavizado de predicciones
prediction_history = deque(maxlen=5) 

# 5. Función para preprocesar el rostro detectado
def preprocess_face(face_img):
    face_img = cv2.resize(face_img, IMG_SIZE)
    img_array = image.img_to_array(face_img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalizar como en el entrenamiento
    return img_array

# 6. Iniciar la cámara web
cap = cv2.VideoCapture(0)

model.predict(np.zeros((1, *IMG_SIZE, 3)))

# 8. Bucle principal de detección
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    # Convertir a escala de grises para detección
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detectar rostros mediante un clasificador Haar de OpenCV
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in faces:
        # Recortar y preprocesar el rostro
        face_img = frame[y:y+h, x:x+w]
        processed_face = preprocess_face(face_img)
        
        # Predecir emoción
        predictions = model.predict(processed_face, verbose=0)
        predicted_class = np.argmax(predictions[0])
        confidence = np.max(predictions[0])
        
        # Suavizar predicciones con historial
        prediction_history.append(predicted_class)
        final_emotion = max(set(prediction_history), key=prediction_history.count)
        
        # Obtener texto para mostrar
        emotion_text = f"{class_names[final_emotion]} ({confidence:.0%})"
        
        # Dibujar rectángulo y texto
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, emotion_text, (x, y-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    # Mostrar el frame resultante
    cv2.imshow('Deteccion de emociones', frame)
    
    # Salir al presionar 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 9. Liberar recursos
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
